In [1]:
library(circlize)
library(rtracklayer)
library(dplyr)
library(GenomicRanges)
library(ComplexHeatmap) ## legend

circlize version 0.4.16
CRAN page: https://cran.r-project.org/package=circlize
Github page: https://github.com/jokergoo/circlize
Documentation: https://jokergoo.github.io/circlize_book/book/

If you use it in published research, please cite:
Gu, Z. circlize implements and enhances circular visualization
  in R. Bioinformatics 2014.

This message can be suppressed by:
  suppressPackageStartupMessages(library(circlize))


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, r

In [2]:
# Read the custom cytoband file
## bioawk -c fastx '{print $name"\t"1"\t"length($seq)}' hap1.fasta|sed "s/_RagTag//" > hap1.ideogram
custom_cytoband <- read.table("chickpea.ideogram", header = FALSE, stringsAsFactors = FALSE)

# Rename columns for clarity
colnames(custom_cytoband) <- c("chr", "start", "end")
custom_cytoband

chr,start,end
<chr>,<int>,<int>
ca1,1,48359943
ca2,1,36634854
ca3,1,39989001
ca4,1,49191682
ca5,1,48169137
ca6,1,59463898
ca7,1,48961560
ca8,1,16477302


In [3]:
# Load necessary library
library(readr)

In [6]:
# Read the tab-delimited file into a table
qtl_file <- "qtl_markers.txt"
qtl_table <- read_delim(qtl_file, delim = "\t", col_names = FALSE)

# Convert the third column into a list of strings
qtl_chromosomes <- as.character(qtl_table[[3]])
qtl_positions <- as.numeric(qtl_table[[4]])
qtl_labels <- as.character(qtl_table[[1]])


Rows: 8 Columns: 4
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): X1, X2, X3
dbl (1): X4

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [7]:
qtl_table

X1,X2,X3,X4
<chr>,<chr>,<chr>,<dbl>
CaAlt1-1,AVR-Ca-00482.01-000775606,ca1,775606
CaAlt3-1,AVR-Ca-01157.03-006626764,ca3,6626764
CaAlt4-1,AVR-Ca-01677.04-031071748,ca4,31071748
CaAlt4-2,AVR-Ca-02085.04-045196212,ca4,45196212
CaAlt5-1,AVR-Ca-02222.05-013946920,ca5,13946920
CaAlt6-1,AVR-Ca-03067.06-035734304,ca6,35734304
CaAlt7-1,AVR-Ca-04258.07-004969265,ca7,4969265
CaAlt7-2,AVR-Ca-04113.07-037532769,ca7,37532769


In [56]:
gene_labels0

[1] "ID=gene-LOC101490635" "ID=gene-LOC101499513" "ID=gene-LOC101497446"
 [4] "ID=gene-LOC101498100" "ID=gene-LOC101498100" "ID=gene-LOC101497782"
 [7] "ID=gene-LOC101506667" "ID=gene-LOC101507327" "ID=gene-LOC101496905"
[10] "ID=gene-LOC101493885" "ID=gene-LOC101494212" "ID=gene-LOC101506246"
[13] "ID=gene-LOC101513987" "ID=gene-LOC101495525" "ID=gene-LOC101505977"
[16] "ID=gene-LOC101488282" "ID=gene-LOC101490040" "ID=gene-LOC101490040"
[19] "ID=gene-LOC101489402" "ID=gene-LOC101489402" "ID=gene-LOC101515427"
[22] "ID=gene-LOC101509284" "ID=gene-LOC101499791" "ID=gene-LOC101509308"
[25] "ID=gene-LOC101491258" "ID=gene-LOC101491258" "ID=gene-LOC101489317"
[28] "ID=gene-LOC101495795" "ID=gene-LOC101515446" "ID=gene-LOC101515446"
[31] "ID=gene-LOC101493863" "ID=gene-LOC101510069" "ID=gene-LOC101514601"
[34] "ID=gene-LOC101514932" "ID=gene-LOC101515259" "ID=gene-LOC101505909"
[37] "ID=gene-LOC101508803" "ID=gene-LOC101514527" "ID=gene-LOC101494238"
[40] "ID=gene-LOC101506246" "ID=gene-LOC101499812" "ID=gene-LOC101515278"
[43] "ID=gene-LOC101498100" "ID=gene-LOC101499812" "ID=gene-LOC101498870"
[46] "ID=gene-LOC101490040" "ID=gene-LOC101499191"

In [59]:
# Read the tab-delimited file into a table
gene_file <- "homolog_gene_less.txt" ## remove gene on chrUn
gene_table <- read_delim(gene_file, delim = "\t")

# Convert the third column into a list of strings
gene_chromosomes <- as.character(gene_table[[2]])
gene_positions <- as.numeric(gene_table[[5]])
# Read the ninth column
ninth_column <- gene_table[[10]]

# Split each entry by ";" and extract the first field
gene_labels0 <- sapply(strsplit(as.character(ninth_column), ";"), function(x) x[1])
# Perform the string replacement
gene_labels <- gsub("ID=gene-LOC101", "C", gene_labels0)


New names:
• `.` -> `....7`
• `.` -> `....9`
Rows: 47 Columns: 10
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (8): XP_004485943.1  LOC101490316, ca1, Gnomon, gene, ....7, +, ....9, I...
dbl (2): 3574531, 3576813

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [60]:
gene_labels

[1] "C490635" "C499513" "C497446" "C498100" "C498100" "C497782" "C506667"
 [8] "C507327" "C496905" "C493885" "C494212" "C506246" "C513987" "C495525"
[15] "C505977" "C488282" "C490040" "C490040" "C489402" "C489402" "C515427"
[22] "C509284" "C499791" "C509308" "C491258" "C491258" "C489317" "C495795"
[29] "C515446" "C515446" "C493863" "C510069" "C514601" "C514932" "C515259"
[36] "C505909" "C508803" "C514527" "C494238" "C506246" "C499812" "C515278"
[43] "C498100" "C499812" "C498870" "C490040" "C499191"

In [43]:
?circos.genomicAxis()

In [66]:
head(breaks)

[1] 0.0e+00 5.0e+06 1.0e+07 1.5e+07 2.0e+07 2.5e+07

In [81]:
gene_blue <- rep("blue", 47)
qtl_red <- rep("red", 8)

In [76]:
length(gene_labels)

[1] 47

In [75]:
length(qtl_labels)

[1] 8

In [80]:
c(gene_blue,qtl_red)

[1] "blue" "blue" "blue" "blue" "blue" "blue" "blue" "blue" "blue" "blue"
[11] "blue" "blue" "blue" "blue" "blue" "blue" "blue" "blue" "blue" "blue"
[21] "blue" "blue" "blue" "blue" "blue" "blue" "blue" "blue" "blue" "blue"
[31] "blue" "blue" "blue" "blue" "blue" "blue" "blue" "blue" "blue" "blue"
[41] "blue" "blue" "blue" "blue" "blue" "blue" "blue" "blue" "blue" "blue"
[51] "blue" "blue" "blue" "blue" "blue"

In [87]:
png("chickpea_circos_updated.png", width = 1000, height = 1000)
# Initialize the circos plot with the custom ideogram
# Set the start angle
circos.par(start.degree = 90)
circos.initializeWithIdeogram(custom_cytoband, plotType = NULL)

circos.labels(c(gene_chromosomes, qtl_chromosomes), x = c(gene_positions,qtl_positions), 
              cex = 1, labels = c(gene_labels,qtl_labels), col = c(gene_blue,qtl_red),
              labels_height = 0.1,
              side = c("outside"))
circos.track(ylim = c(0, 1), track.height = 0.1, panel.fun = function(x, y) {
    breaks = seq(0, 1e9, by = 1e7)
    circos.genomicAxis(h = "top",major.at = breaks, labels = paste0(breaks/1e6, ""), 
            direction = "inside", labels.cex = 1)
    circos.text(CELL_META$xcenter, - (CELL_META$ycenter + 0.2), CELL_META$sector.index, cex=1.5)

})


circos.clear()# Close the PNG device
dev.off()

Note: 1 point is out of plotting region in sector 'ca1', track '3'.

Note: 1 point is out of plotting region in sector 'ca2', track '3'.

Note: 1 point is out of plotting region in sector 'ca3', track '3'.

Note: 1 point is out of plotting region in sector 'ca4', track '3'.

Note: 1 point is out of plotting region in sector 'ca5', track '3'.

Note: 1 point is out of plotting region in sector 'ca6', track '3'.

Note: 1 point is out of plotting region in sector 'ca7', track '3'.

Note: 1 point is out of plotting region in sector 'ca8', track '3'.



pdf 
  2

In [84]:
?circos.genomicAxis()

In [34]:
?circos.trackPlotRegion()

In [36]:
?circos.labels()

In [70]:
png("chickpea_circos_updated.png", width = 800, height = 800)
# Initialize the circos plot with the custom ideogram
# Set the start angle
circos.par(start.degree = 90)
circos.initializeWithIdeogram(custom_cytoband, plotType = NULL)

circos.track(ylim = c(0, 1), track.height = 0.1, panel.fun = function(x, y) {
    breaks = seq(0, 1e9, by = 5e6)
    circos.genomicAxis(major.at = breaks, labels = paste0(breaks/1e6, ""), labels.cex = 1)
    circos.text(CELL_META$xcenter, CELL_META$ycenter, CELL_META$sector.index, cex=1.5)

})

circos.track(ylim = c(0, 1), track.height = 0.05)
circos.labels(c(gene_chromosomes, qtl_chromosomes), x = c(gene_positions,qtl_positions), 
              cex = 1.2, labels = c(gene_labels,qtl_labels))

circos.clear()# Close the PNG device
dev.off()

pdf 
  2

In [71]:
gene_table

XP_004486970.1 LOC101497782,ca1,Gnomon,gene,11155685,11160403,....7,+,....9,ID=gene-LOC101497782;Dbxref=GeneID:101497782;Name=LOC101497782;gbkey=Gene;gene=LOC101497782;gene_biotype=protein_coding
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
XP_004487722.1 LOC101493885,ca1,Gnomon,gene,14463723,14466287,.,+,.,ID=gene-LOC101493885;Dbxref=GeneID:101493885;Name=LOC101493885;gbkey=Gene;gene=LOC101493885;gene_biotype=protein_coding
XP_004491273.1 LOC101513987,ca2,Gnomon,gene,33781947,33784917,.,+,.,ID=gene-LOC101513987;Dbxref=GeneID:101513987;Name=LOC101513987;gbkey=Gene;gene=LOC101513987;gene_biotype=protein_coding
XP_004493719.1 LOC101495525,ca3,Gnomon,gene,30493076,30496933,.,-,.,ID=gene-LOC101495525;Dbxref=GeneID:101495525;Name=LOC101495525;gbkey=Gene;gene=LOC101495525;gene_biotype=protein_coding
XP_004494917.1 LOC101488282,ca3,Gnomon,gene,35680003,35684082,.,-,.,ID=gene-LOC101488282;Dbxref=GeneID:101488282;Name=LOC101488282;gbkey=Gene;gene=LOC101488282;gene_biotype=protein_coding
XP_004499881.1 LOC101509308,ca5,Gnomon,gene,16522379,16529693,.,+,.,ID=gene-LOC101509308;Dbxref=GeneID:101509308;Name=LOC101509308;gbkey=Gene;gene=LOC101509308;gene_biotype=protein_coding
XP_004506221.1 LOC101505909,ca6,Gnomon,gene,31633696,31636481,.,-,.,ID=gene-LOC101505909;Dbxref=GeneID:101505909;Name=LOC101505909;gbkey=Gene;gene=LOC101505909;gene_biotype=protein_coding
XP_004510955.1 LOC101514527,ca7,Gnomon,gene,39786461,39798690,.,-,.,ID=gene-LOC101514527;Dbxref=GeneID:101514527;Name=LOC101514527;gbkey=Gene;gene=LOC101514527;gene_biotype=protein_coding
